In [712]:
class Singleton(object):
    ans = None

    @staticmethod
    def instance():
        if '_instance' not in Singleton.__dict__:
            Singleton._instance = Singleton()
        return Singleton._instance

In [713]:
s1 = Singleton.instance()
s2 = Singleton.instance()

assert s1 is s2

s1.ans = 42

assert s2.ans == s1.ans

In [714]:
print(s1)
print(s2)

In [715]:
import datetime
import os


class MetaSingleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(MetaSingleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Logger(metaclass=MetaSingleton):
    now = datetime.datetime.now()
    date_format, time_format = '%d.%m.%Y %H:%M:%S'.split()

    def __init__(self, directory=None, filename=None):
        self.__log_file = self.create_log_file(directory, filename)

    def __get_date(self):
        return self.__class__.now.strftime(self.__class__.date_format)

    def __get_time(self):
        return self.__class__.now.strftime(self.__class__.time_format)

    def get_file_path(self, directory=None, filename=None):
        if not directory:
            directory = os.getcwd()
        if not filename:
            filename = '%s.log' % self.__get_date()
        file_path = os.path.join(directory, filename)
        return file_path

    def create_log_file(self, directory=None, filename=None):
        full_path = self.get_file_path(directory, filename)
        try:
            log_file = open(full_path, 'w').close()
            return log_file
        except Exception as e:
            raise RuntimeError("Unable to open log file")

    def write_log(self, log_record):
        now = str(self.__get_time())
        record = '%s: %s\n' % (now, log_record)
        with open(self.get_file_path(), 'a') as file:
            file.write(record)

    def clear_log(self):
        open(self.get_file_path(), 'w').close()

    def get_logs(self):
        with open(self.get_file_path(), 'r') as file:
            lines = file.read().splitlines()
            return lines

    def get_last_event(self):
        try:
            return self.get_logs()[-1]
        except Exception as e:
            return 'This file is empty', e

    @staticmethod
    def get_all_logs(directory=None):
        file_list = []
        if not directory:
            directory = os.getcwd()
        for file in os.listdir(directory):
            if file.endswith('.log'):
                file_list.append(file)
        return file_list

In [716]:
log1 = Logger()
log2 = Logger()

print(log1==log2)

True


In [717]:
log1.write_log('hello!')
log1.write_log('hello world!')

In [718]:
log1.get_logs()

['05:38:27: hello!', '05:38:27: hello world!']

In [719]:
log1.get_last_event()

'05:38:27: hello world!'

In [720]:
log1.get_all_logs()

['29.08.2022 — копия.log', '29.08.2022.log']

In [721]:
log1.clear_log()

In [722]:
log1.write_log('hello!')
log1.write_log('hello world!')

In [723]:
log1.get_logs()

['05:38:27: hello!', '05:38:27: hello world!']